In [3]:
import sys
!{sys.executable} -m pip install --user --upgrade pip
!{sys.executable} -m pip install --user requests geopy gtfslite
!{sys.executable} -m pip install --user --upgrade gtfs-kit pandas pyproj

  Using cached geopy-2.4.1-py3-none-any.whl.metadata (6.8 kB)


ERROR: Could not find a version that satisfies the requirement gtfslite (from versions: none)
ERROR: No matching distribution found for gtfslite


In [5]:
import gtfs_kit as gk
path = r"C:\Users\ts261157\Desktop\idfm_gtfs.zip"
feed = gk.read_feed(path, dist_units="km")
print("Feed loaded successfully with", len(feed.stops), "stops")

C:\Users\ts261157\AppData\Roaming\Python\Python313\site-packages\gtfs_kit\feed.py:392: DtypeWarning: Columns (12,13) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(


Feed loaded successfully with 53564 stops


In [6]:
# Keep only metro routes (route_type = 1)
metro_routes = feed.routes.query("route_type == 1")
metro_trips = feed.trips[feed.trips.route_id.isin(metro_routes.route_id)]
metro_stop_times = feed.stop_times[feed.stop_times.trip_id.isin(metro_trips.trip_id)]

print("Metro routes:", len(metro_routes))
print("Metro trips:", len(metro_trips))
print("Stop-time records:", len(metro_stop_times))


C:\Users\ts261157\AppData\Local\Temp\ipykernel_17728\1779791558.py:2: RuntimeWarning: Engine has switched to 'python' because numexpr does not support extension array dtypes. Please set your engine to python manually.
  metro_routes = feed.routes.query("route_type == 1")


Metro routes: 16
Metro trips: 47123
Stop-time records: 1214582


In [8]:
import pandas as pd

def gtfs_time_to_seconds(t):
    """Convert 'HH:MM:SS' strings (including >24h) to total seconds."""
    if pd.isna(t):
        return None
    h, m, s = map(int, str(t).split(":"))
    return h * 3600 + m * 60 + s


In [15]:
# Pick the two stations again
stops = feed.stops
stop_a = stops[stops.stop_name == "Châtelet les Halles"].iloc[0]
stop_b = stops[stops.stop_name == "Nation"].iloc[0]

# Limit to Line 1
line1 = metro_routes[metro_routes.route_short_name == "1"]
trips_line1 = metro_trips[metro_trips.route_id.isin(line1.route_id)]
stop_times_line1 = metro_stop_times[metro_stop_times.trip_id.isin(trips_line1.trip_id)]

# Extract stop times for both stations
st_a = stop_times_line1[stop_times_line1.stop_id == stop_a.stop_id]
st_b = stop_times_line1[stop_times_line1.stop_id == stop_b.stop_id]
merged = pd.merge(st_a, st_b, on="trip_id", suffixes=("_a", "_b"))
merged = merged[merged.stop_sequence_a < merged.stop_sequence_b]

# Convert string times → seconds
merged["dep_a_sec"] = merged["departure_time_a"].apply(gtfs_time_to_seconds)
merged["arr_b_sec"] = merged["arrival_time_b"].apply(gtfs_time_to_seconds)

# Compute travel time in minutes
merged["travel_min"] = (merged["arr_b_sec"] - merged["dep_a_sec"]) / 60
merged = merged[merged.travel_min > 0]  # remove invalid negatives

print(f"Approximate travel time Châtelet → Nation: {merged.travel_min.min():.1f} minutes")


Approximate travel time Châtelet → Nation: <NA> minutes


In [14]:
# All Châtelet and Nation stops in your GTFS feed
stops = feed.stops

print("\nChâtelet variants:")
display(stops[stops.stop_name.str.contains("Châtelet", case=False, na=False)][["stop_id","stop_name"]])

print("\nNation variants:")
display(stops[stops.stop_name.str.contains("Nation", case=False, na=False)][["stop_id","stop_name"]])



Châtelet variants:


,stop_id,stop_name
13,IDFM:monomodalStopPlace:45102,Châtelet les Halles
1422,IDFM:14882,Avenue du Châtelet
2205,IDFM:492289,Châtelet
3689,IDFM:493026,Châtelet
6353,IDFM:483442,Châtelet
6765,IDFM:486056,Châtelet
7108,IDFM:482774,Châtelet
7940,IDFM:426200,Châtelet
10815,IDFM:479434,Châtelets
11339,IDFM:41401,Châtelet



Nation variants:


,stop_id,stop_name
0,IDFM:monomodalStopPlace:473875,Nation
1028,IDFM:28674,Bibliothèque Nationale
1575,IDFM:30774,Collège International
1751,IDFM:24022,Bibliothèque Nationale de France
1752,IDFM:24021,Bibliothèque Nationale de France
...,...,...
50835,IDFM:69263,Route Nationale
51428,IDFM:StopPlaceEntrance:50170399,Avenue des Nations
53154,IDFM:StopPlaceEntrance:50148082,Caisse Nationale d'Assurance Vieillesse
53197,IDFM:StopPlaceEntrance:50148695,École Nationale Vétérinaire


In [16]:
# Find trips that stop at each station on Metro Line 1
stops_line1 = metro_stop_times.merge(metro_trips, on="trip_id").merge(metro_routes, on="route_id")

chatelet_trips = stops_line1.query("stop_id == @stop_a.stop_id")["trip_id"].unique()
nation_trips = stops_line1.query("stop_id == @stop_b.stop_id")["trip_id"].unique()

print("Trips serving Châtelet (M1):", len(chatelet_trips))
print("Trips serving Nation (M1):", len(nation_trips))


Trips serving Châtelet (M1): 0
Trips serving Nation (M1): 0


In [17]:
routes_line1 = feed.routes[feed.routes.route_short_name == "1"]
print(routes_line1[["route_id","route_long_name","route_desc"]])


         route_id route_long_name route_desc
9     IDFM:C00324               1       <NA>
177   IDFM:C01423               1       <NA>
467   IDFM:C01371               1       <NA>
681   IDFM:C00292               1       <NA>
963   IDFM:C00189               1       <NA>
1428  IDFM:C01826               1       <NA>


In [18]:
for rid in ["IDFM:C00324", "IDFM:C01423", "IDFM:C01371", "IDFM:C00292", "IDFM:C00189", "IDFM:C01826"]:
    trips_line1 = feed.trips[feed.trips.route_id == rid]
    stop_times_line1 = feed.stop_times[feed.stop_times.trip_id.isin(trips_line1.trip_id)]
    stops_line1 = feed.stops[feed.stops.stop_id.isin(stop_times_line1.stop_id)]
    names = list(stops_line1.stop_name.unique())
    print(f"\nRoute {rid}: {len(names)} stops")
    print([n for n in names if "Chât" in n or "Nation" in n])



Route IDFM:C00324: 47 stops
[]

Route IDFM:C01423: 28 stops
[]

Route IDFM:C01371: 25 stops
['Château de Vincennes', 'Nation', 'Châtelet']

Route IDFM:C00292: 31 stops
[]

Route IDFM:C00189: 41 stops
[]

Route IDFM:C01826: 24 stops
[]


In [20]:
import pandas as pd

route_id = "IDFM:C01371"  # working variant for Métro 1
trips_line1 = feed.trips[feed.trips.route_id == route_id]
stop_times_line1 = feed.stop_times[feed.stop_times.trip_id.isin(trips_line1.trip_id)]

# Stops actually used on this route
stops_line1 = feed.stops[feed.stops.stop_id.isin(stop_times_line1.stop_id)]

stop_a = stops_line1[stops_line1.stop_name.str.contains("Châtelet", case=False, na=False)].iloc[0]
stop_b = stops_line1[stops_line1.stop_name.str.contains("Nation", case=False, na=False)].iloc[0]

print("Using stops:", stop_a.stop_name, "→", stop_b.stop_name)


Using stops: Châtelet → Nation


In [21]:
def gtfs_time_to_seconds(t):
    if pd.isna(t): 
        return None
    h, m, s = map(int, str(t).split(":"))
    return h * 3600 + m * 60 + s

st_a = stop_times_line1[stop_times_line1.stop_id == stop_a.stop_id]
st_b = stop_times_line1[stop_times_line1.stop_id == stop_b.stop_id]

merged = pd.merge(st_a, st_b, on="trip_id", suffixes=("_a", "_b"))
merged = merged[merged.stop_sequence_a < merged.stop_sequence_b]

merged["dep_a_sec"] = merged["departure_time_a"].apply(gtfs_time_to_seconds)
merged["arr_b_sec"] = merged["arrival_time_b"].apply(gtfs_time_to_seconds)
merged["travel_min"] = (merged["arr_b_sec"] - merged["dep_a_sec"]) / 60
merged = merged[merged.travel_min > 0]

if not merged.empty:
    print(f"Approximate travel time {stop_a.stop_name} → {stop_b.stop_name}: {merged.travel_min.min():.1f} minutes")
else:
    print("No matching trips on this route variant.")


Approximate travel time Châtelet → Nation: 9.0 minutes
